In [60]:
import pandas as pd
import psycopg2
import csv

In [3]:
jiggy_ecom = pd.read_csv(r'C:\Users\HP\Desktop\New_class\case_study\jiggy_ecommerce_sales_transactions_with_customer_details.csv')

In [4]:
jiggy_ecom.head()

,Date,ProductName,Category,Quantity,UnitPrice,TotalPrice,PaymentType,Country,City,PostalCode,...,ProductReturnDate,ReturnReason,RefundAmount,CustomerSupportContact,PromotionApplied,CouponCode,MembershipStatus,ReviewComment,ReviewDate,FollowUpAction
0,2021-01-01,JBSKMJNRSVIYG,Home & Kitchen,5,250.88,1254.40,PayPal,France,Paris,40804,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-01,TFUBFCP,Books,8,250.88,2007.04,Bank Transfer,UK,New York,75206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-02,BQILCKAWMOZE,Clothing,1,250.88,250.88,Bank Transfer,France,Toronto,20934,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-02,QYWXYFUVK,Toys & Games,1,250.88,250.88,Bank Transfer,Canada,New York,40723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-04,DCMWIU,Electronics,9,250.88,2257.92,PayPal,France,Sydney,30405,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
jiggy_ecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 46 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Category                1000 non-null   object 
 3   Quantity                1000 non-null   int64  
 4   UnitPrice               1000 non-null   float64
 5   TotalPrice              1000 non-null   float64
 6   PaymentType             1000 non-null   object 
 7   Country                 1000 non-null   object 
 8   City                    1000 non-null   object 
 9   PostalCode              1000 non-null   int64  
 10  ShippingCost            1000 non-null   float64
 11  OrderStatus             1000 non-null   object 
 12  ReturnStatus            1000 non-null   object 
 13  Rating                  1000 non-null   int64  
 14  Feedback                1000 non-null   o

In [6]:
jiggy_ecom.columns

Index(['Date', 'ProductName', 'Category', 'Quantity', 'UnitPrice',
       'TotalPrice', 'PaymentType', 'Country', 'City', 'PostalCode',
       'ShippingCost', 'OrderStatus', 'ReturnStatus', 'Rating', 'Feedback',
       'ShippingDate', 'DeliveryDate', 'CustomerFullName', 'CustomerPhone',
       'CustomerEmail', 'CustomerAddress', 'ProductDescription',
       'SupplierName', 'SupplierContact', 'PurchasePrice', 'Profit',
       'Discount', 'Tax', 'TaxAmount', 'FinalPrice', 'PaymentStatus',
       'ShipmentTrackingNumber', 'Carrier', 'EstimatedDelivery',
       'ActualDelivery', 'DeliveryStatus', 'ProductReturnDate', 'ReturnReason',
       'RefundAmount', 'CustomerSupportContact', 'PromotionApplied',
       'CouponCode', 'MembershipStatus', 'ReviewComment', 'ReviewDate',
       'FollowUpAction'],
      dtype='object')

In [7]:
customer = jiggy_ecom[['CustomerFullName', 'CustomerPhone', 'CustomerEmail', 'CustomerAddress','Country',
                      'City', 'PostalCode']].copy().drop_duplicates().reset_index(drop=True)
customer['customer_id'] = range(1,len(customer)+1)
customer = customer[['customer_id','CustomerFullName', 'CustomerPhone',
       'CustomerEmail', 'CustomerAddress','Country', 'City', 'PostalCode']]
customer.head()

,customer_id,CustomerFullName,CustomerPhone,CustomerEmail,CustomerAddress,Country,City,PostalCode
0,1,HBNPTJREPKSFGV,1.401023e+10,unorc@example.com,LCJFUQABSTZFJJRWHTGJYAVOJVC,France,Paris,40804
1,2,RTWHUJQIMHQ,1.311776e+10,mcgsw@example.com,IMFSBLKMVCQIWOODNNNRC,UK,New York,75206
2,3,PUHZJLRWIWYDEWPARR,1.044630e+10,xrqvi@example.com,HSBEXIKPTBGEYAUDIORJ,France,Toronto,20934
3,4,YSIACZBBQFEUKIVYDGM,1.719314e+10,nvhne@example.com,KQZRLWMRVQJOTWXHUKRDSWVBTC,Canada,New York,40723
4,5,QTBERMRKNLDGM,NaN,xtnlv@example.com,QLSYJTMNPZXAXRC,France,Sydney,30405


In [8]:
shipping = jiggy_ecom[['ShippingCost', 'OrderStatus', 'ReturnStatus', 'Rating', 'Feedback',
       'ShippingDate', 'DeliveryDate']].copy().drop_duplicates().reset_index(drop=True)
shipping['shipping_id'] = range(1, len(shipping) +1)
shipping = shipping[['shipping_id','ShippingCost', 'OrderStatus', 'ReturnStatus', 'Rating', 'Feedback',
       'ShippingDate', 'DeliveryDate']]

shipping.head()

,shipping_id,ShippingCost,OrderStatus,ReturnStatus,Rating,Feedback,ShippingDate,DeliveryDate
0,1,34.42,Refunded,Returned,3,QAAGDWULPNWJNGDBXRPIEIYHXTPT,2021-01-05,2021-01-12
1,2,34.42,Refunded,Not Returned,2,RDPDFCROHRXWCFVZUXOLEGVOGMYCZWMZCBWDQFKFVBQCLNBVV,2021-01-03,2021-01-12
2,3,34.42,Completed,Not Returned,4,EJJUIUMJFKDZNQTBJCGGAHWAHNNBHSJGMVUOKZNKBBLOQNYP,2021-01-06,2021-01-09
3,4,34.42,Completed,Returned,2,HXQELFRZEFIOJQZSLIONAQAWUVMHMPBMRCMSMSDSRYFSYD,2021-01-04,2021-01-10
4,5,34.42,Completed,Returned,3,XGGZWZPPEOWDXYMSGLZIQBP,2021-01-07,2021-01-09


In [9]:
shipping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   shipping_id   1000 non-null   int64  
 1   ShippingCost  1000 non-null   float64
 2   OrderStatus   1000 non-null   object 
 3   ReturnStatus  1000 non-null   object 
 4   Rating        1000 non-null   int64  
 5   Feedback      1000 non-null   object 
 6   ShippingDate  1000 non-null   object 
 7   DeliveryDate  1000 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 62.6+ KB


In [10]:
product = jiggy_ecom[['ProductName','ProductDescription',
       'SupplierName', 'SupplierContact', 'PurchasePrice', 'Profit',
       'Tax', 'TaxAmount', 'FinalPrice', 'PaymentStatus',
       'ShipmentTrackingNumber']].copy().drop_duplicates().reset_index(drop=True)
product['product_id'] = range(1, len(product)+1)
product = product[['product_id','ProductName','ProductDescription',
       'SupplierName', 'SupplierContact', 'PurchasePrice', 'Profit',
       'Tax', 'TaxAmount', 'FinalPrice', 'PaymentStatus',
       'ShipmentTrackingNumber']]
product.head()

,product_id,ProductName,ProductDescription,SupplierName,SupplierContact,PurchasePrice,Profit,Tax,TaxAmount,FinalPrice,PaymentStatus,ShipmentTrackingNumber
0,1,JBSKMJNRSVIYG,NaN,GAZBHXKMERSUC,WLNHIBUBXPTKTJHGA,205.26,228.10,0.07,87.81,1317.1220,Failed,804964.0
1,2,TFUBFCP,NaN,AVESIZXGLNPMRJ,NaN,205.26,364.96,0.07,140.49,1886.6148,Processed,377767.0
2,3,BQILCKAWMOZE,NaN,UFVCDAKQE,XUBVMSAFHQC,205.26,45.62,0.07,17.56,263.4224,Pending,874536.0
3,4,QYWXYFUVK,NaN,WGDZPONB,ZSGOFYVWRJDERJJ,205.26,45.62,0.07,17.56,233.3168,NaN,847385.0
4,5,DCMWIU,NaN,TXBEMWJVMNYRTW,GTNYHZIXEYUHTETRDKU,205.26,410.58,0.07,158.05,2257.9156,Failed,560121.0


In [11]:
product.shape

(1000, 12)

In [12]:
inventory = jiggy_ecom[['Carrier', 'EstimatedDelivery',
       'ActualDelivery', 'DeliveryStatus', 'ProductReturnDate', 'ReturnReason',
       'RefundAmount', 'CustomerSupportContact', 'PromotionApplied',
       'CouponCode', 'MembershipStatus', 'ReviewComment', 'ReviewDate',
       'FollowUpAction']].copy().drop_duplicates().reset_index(drop=True)
inventory['inventory_id'] = range(1,len(inventory)+1)
inventory= inventory[['inventory_id','Carrier', 'EstimatedDelivery',
       'ActualDelivery', 'DeliveryStatus', 'ProductReturnDate', 'ReturnReason',
       'RefundAmount', 'CustomerSupportContact', 'PromotionApplied',
       'CouponCode', 'MembershipStatus', 'ReviewComment', 'ReviewDate',
       'FollowUpAction']]
inventory.head()

,inventory_id,Carrier,EstimatedDelivery,ActualDelivery,DeliveryStatus,ProductReturnDate,ReturnReason,RefundAmount,CustomerSupportContact,PromotionApplied,CouponCode,MembershipStatus,ReviewComment,ReviewDate,FollowUpAction
0,1,UPS,2021-01-12,2021-01-12,On Time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,USPS,2021-01-11,2021-01-12,Early,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,DHL,2021-01-10,2021-01-09,Early,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,UPS,2021-01-08,2021-01-09,Early,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,DHL,2021-01-07,2021-01-08,Delayed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
inventory.shape

(991, 15)

In [14]:
inventory.head()

,inventory_id,Carrier,EstimatedDelivery,ActualDelivery,DeliveryStatus,ProductReturnDate,ReturnReason,RefundAmount,CustomerSupportContact,PromotionApplied,CouponCode,MembershipStatus,ReviewComment,ReviewDate,FollowUpAction
0,1,UPS,2021-01-12,2021-01-12,On Time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,USPS,2021-01-11,2021-01-12,Early,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,DHL,2021-01-10,2021-01-09,Early,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,UPS,2021-01-08,2021-01-09,Early,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,DHL,2021-01-07,2021-01-08,Delayed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
sales = jiggy_ecom.merge(customer, on= ['CustomerFullName', 'CustomerPhone', 'CustomerEmail', 'CustomerAddress','Country',             
                                        'City', 'PostalCode'], how='left') \
                 .merge(shipping, on=['ShippingCost', 'OrderStatus', 'ReturnStatus', 'Rating', 'Feedback',
                                       'ShippingDate', 'DeliveryDate'], how='left') \
                 .merge(product, on=['ProductName','ProductDescription','SupplierName', 'SupplierContact', 'PurchasePrice',
                                       'Profit','Tax', 'TaxAmount', 'FinalPrice', 'PaymentStatus', 'ShipmentTrackingNumber'], how='left') \
                .merge(inventory, on=['Carrier', 'EstimatedDelivery','ActualDelivery', 'DeliveryStatus', 'ProductReturnDate', 'ReturnReason',
                                         'RefundAmount', 'CustomerSupportContact', 'PromotionApplied','CouponCode', 'MembershipStatus', 
                                         'ReviewComment','ReviewDate','FollowUpAction'], how='left') \
                   [['customer_id','shipping_id','inventory_id', 'product_id','Quantity', 'UnitPrice', 'TotalPrice','PaymentType','Discount' ]]                      
sales['sales_id'] = range(1, len(sales) +1)
sales = sales[['sales_id','customer_id','shipping_id','inventory_id', 'product_id','Quantity', 'UnitPrice', 'TotalPrice','PaymentType','Discount']]


sales.head()

,sales_id,customer_id,shipping_id,inventory_id,product_id,Quantity,UnitPrice,TotalPrice,PaymentType,Discount
0,1,1,1,1,1,5,250.88,1254.40,PayPal,0.02
1,2,2,2,2,2,8,250.88,2007.04,Bank Transfer,0.13
2,3,3,3,3,3,1,250.88,250.88,Bank Transfer,0.02
3,4,4,4,4,4,1,250.88,250.88,Bank Transfer,0.14
4,5,5,5,5,5,9,250.88,2257.92,PayPal,0.07


In [16]:
# cleaning the data
product.info()
product.fillna({
   'ProductDescription': 'Not given',
   'SupplierContact': 'Unknown',
   'PaymentStatus': 'Unknown',
   'ShipmentTrackingNumber': 0.0
}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_id              1000 non-null   int64  
 1   ProductName             1000 non-null   object 
 2   ProductDescription      0 non-null      float64
 3   SupplierName            1000 non-null   object 
 4   SupplierContact         900 non-null    object 
 5   PurchasePrice           1000 non-null   float64
 6   Profit                  1000 non-null   float64
 7   Tax                     1000 non-null   float64
 8   TaxAmount               1000 non-null   float64
 9   FinalPrice              1000 non-null   float64
 10  PaymentStatus           900 non-null    object 
 11  ShipmentTrackingNumber  900 non-null    float64
dtypes: float64(7), int64(1), object(4)
memory usage: 93.9+ KB


In [17]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_id              1000 non-null   int64  
 1   ProductName             1000 non-null   object 
 2   ProductDescription      1000 non-null   object 
 3   SupplierName            1000 non-null   object 
 4   SupplierContact         1000 non-null   object 
 5   PurchasePrice           1000 non-null   float64
 6   Profit                  1000 non-null   float64
 7   Tax                     1000 non-null   float64
 8   TaxAmount               1000 non-null   float64
 9   FinalPrice              1000 non-null   float64
 10  PaymentStatus           1000 non-null   object 
 11  ShipmentTrackingNumber  1000 non-null   float64
dtypes: float64(6), int64(1), object(5)
memory usage: 93.9+ KB


In [18]:
customer.info()
customer.fillna({
   'CustomerPhone':0.0,
   'CustomerEmail': 'Unknown' 
}, inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       1000 non-null   int64  
 1   CustomerFullName  1000 non-null   object 
 2   CustomerPhone     900 non-null    float64
 3   CustomerEmail     900 non-null    object 
 4   CustomerAddress   1000 non-null   object 
 5   Country           1000 non-null   object 
 6   City              1000 non-null   object 
 7   PostalCode        1000 non-null   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 62.6+ KB


In [19]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       1000 non-null   int64  
 1   CustomerFullName  1000 non-null   object 
 2   CustomerPhone     1000 non-null   float64
 3   CustomerEmail     1000 non-null   object 
 4   CustomerAddress   1000 non-null   object 
 5   Country           1000 non-null   object 
 6   City              1000 non-null   object 
 7   PostalCode        1000 non-null   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 62.6+ KB


In [20]:
inventory.info()
inventory.fillna({
    'ProductReturnDate':0.0,
    'ReturnReason': 0.0,
    'RefundAmount': 0.0,
    'CustomerSupportContact': 0.0,
    'PromotionApplied': 0.0,
    'CouponCode': 0.0,
    'MembershipStatus': 0.0,
    'ReviewComment': 0.0,
    'ReviewDate': 0.0,
    'FollowUpAction' : 0.0
}, inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   inventory_id            991 non-null    int64  
 1   Carrier                 991 non-null    object 
 2   EstimatedDelivery       991 non-null    object 
 3   ActualDelivery          991 non-null    object 
 4   DeliveryStatus          991 non-null    object 
 5   ProductReturnDate       0 non-null      float64
 6   ReturnReason            0 non-null      float64
 7   RefundAmount            0 non-null      float64
 8   CustomerSupportContact  0 non-null      float64
 9   PromotionApplied        0 non-null      float64
 10  CouponCode              0 non-null      float64
 11  MembershipStatus        0 non-null      float64
 12  ReviewComment           0 non-null      float64
 13  ReviewDate              0 non-null      float64
 14  FollowUpAction          0 non-null      fl

In [21]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   inventory_id            991 non-null    int64  
 1   Carrier                 991 non-null    object 
 2   EstimatedDelivery       991 non-null    object 
 3   ActualDelivery          991 non-null    object 
 4   DeliveryStatus          991 non-null    object 
 5   ProductReturnDate       991 non-null    float64
 6   ReturnReason            991 non-null    float64
 7   RefundAmount            991 non-null    float64
 8   CustomerSupportContact  991 non-null    float64
 9   PromotionApplied        991 non-null    float64
 10  CouponCode              991 non-null    float64
 11  MembershipStatus        991 non-null    float64
 12  ReviewComment           991 non-null    float64
 13  ReviewDate              991 non-null    float64
 14  FollowUpAction          991 non-null    fl

In [22]:
#saving to memory
customer.to_csv(r'C:\Users\HP\Desktop\New_class\case_study\dataset\customer.csv', index=False)
product.to_csv(r'C:\Users\HP\Desktop\New_class\case_study\dataset\product.csv', index=False)
inventory.to_csv(r'C:\Users\HP\Desktop\New_class\case_study\dataset\inventory.csv', index=False)
sales.to_csv(r'C:\Users\HP\Desktop\New_class\case_study\dataset\sales.csv', index=False)
shipping.to_csv(r'C:\Users\HP\Desktop\New_class\case_study\dataset\shipping.csv', index=False)


In [ ]:
# Loading the data to the database (PG Admin)
# method 1. psycopg2
# method 2. sqlchemy


## Loadind Data using psycopg2

#

In [41]:
def get_db_connection():
    connection = psycopg2.connect(
        host='localhost',
        database= 'practice',
        user='postgres',
        password='protocol147'
    )
    return connection

In [42]:
conn= get_db_connection()

In [58]:
def create_tables():
    conn= get_db_connection()
    cursor = conn.cursor()
    create_table_query = '''
                          CREATE SCHEMA IF NOT EXISTS jiggy;

                          DROP TABLE IF EXISTS jiggy.customer CASCADE;
                          DROP TABLE IF EXISTS jiggy.shipping CASCADE;
                          DROP TABLE IF EXISTS jiggy.inventory CASCADE;
                          DROP TABLE IF EXISTS jiggy.product CASCADE;
                          DROP TABLE IF EXISTS jiggy.sales CASCADE;

                          CREATE TABLE IF NOT EXISTS jiggy.customer(
                                customer_id SERIAL PRIMARY KEY,
                                CustomerFullName VARCHAR(1000),
                                CustomerPhone FLOAT,
                                CustomerEmail  VARCHAR(1000),
                                CustomerAddress VARCHAR(1000),
                                Country VARCHAR(1000),
                                City VARCHAR(1000),
                                PostalCode INTEGER);
                            
                            
                            CREATE TABLE IF NOT EXISTS jiggy.product(
                                product_id SERIAL PRIMARY KEY,
                                ProductName VARCHAR(1000),
                                ProductDescription VARCHAR(1000),
                                SupplierName  VARCHAR(1000),
                                SupplierContact VARCHAR(1000),
                                PurchasePrice FLOAT,
                                Profit FLOAT,
                                Tax FLOAT,
                                TaxAmount FLOAT,
                                FinalPrice FLOAT,
                                PaymentStatus VARCHAR(1000),
                                ShipmentTrackingNumber FLOAT);
                            
                            
                            CREATE TABLE IF NOT EXISTS jiggy.shipping(
                                shipping_id SERIAL PRIMARY KEY,
                                ShippingCost FLOAT,
                                OrderStatus VARCHAR(1000),
                                ReturnStatus  VARCHAR(1000),
                                Rating INTEGER,
                                Feedback VARCHAR(1000),
                                ShippingDate VARCHAR(1000),
                                DeliveryDate VARCHAR(1000));
                            
                            CREATE TABLE IF NOT EXISTS jiggy.inventory(
                                inventory_id SERIAL PRIMARY KEY,
                                Carrier VARCHAR(1000),
                                EstimatedDelivery VARCHAR(1000),
                                ActualDelivery VARCHAR(1000),
                                DeliveryStatus VARCHAR(1000),
                                ProductReturnDate FLOAT,
                                ReturnReason FLOAT,
                                RefundAmount FLOAT,
                                CustomerSupportContact FLOAT,
                                PromotionApplied FLOAT,
                                CouponCode FLOAT,
                                MembershipStatus FLOAT,
                                ReviewComment FLOAT,
                                ReviewDate FLOAT,
                                FollowUpAction FLOAT
                            );
                            
                            CREATE TABLE IF NOT EXISTS jiggy.inventory(
                                sales_id SERIAL PRIMARY KEY,
                                customer_id INTEGER,
                                shipping_id INTEGER,
                                inventory_id INTEGER,
                                product_id INTEGER,
                                Quantity INTEGER,
                                UnitPrice FLOAT,
                                TotalPrice FLOAT,
                                PaymentType VARCHAR(1000),
                                Discount FLOAT
                            );
                            
                            
                            '''
                            
                            

    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

In [61]:
create_tables()

In [66]:
## Load csv files to DataBase 

def load_product_csv(csv_path):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
            '''INSERT INTO jiggy.product (product_id, ProductName, ProductDescription, SupplierName,SupplierContact, PurchasePrice, Profit, Tax, TaxAmount,
               FinalPrice, PaymentStatus, ShipmentTrackingNumber) 
               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)''', 
               row 
               )
        
    conn.commit()
    cursor.close()
    conn.close()   

csv_path = r'C:\Users\HP\Desktop\New_class\case_study\dataset\product.csv'

load_product_csv(csv_path)
print('product data has been loaded successfully')
        

product data has been loaded successfully


In [69]:
def load_customer_csv(csv_path):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
            '''INSERT INTO jiggy.customer (customer_id, CustomerFullName, CustomerPhone, CustomerEmail,
               CustomerAddress, Country, City, PostalCode) 
               VALUES (%s,%s,%s,%s,%s,%s,%s,%s)''', 
               row 
               )
        
    conn.commit()
    cursor.close()
    conn.close() 

    


In [71]:
csv_path = r'C:\Users\HP\Desktop\New_class\case_study\dataset\customer.csv'

load_customer_csv(csv_path)
print('customer data has been loaded successfully')

customer data has been loaded successfully


In [75]:
def load_shipping_csv(csv_path):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
            '''INSERT INTO jiggy.shipping (shipping_id, ShippingCost, OrderStatus, ReturnStatus, Rating,
               Feedback, ShippingDate, DeliveryDate) 
               VALUES (%s,%s,%s,%s,%s,%s,%s,%s)''', 
               row 
               )
        
    conn.commit()
    cursor.close()
    conn.close()

In [76]:
csv_path = r'C:\Users\HP\Desktop\New_class\case_study\dataset\shipping.csv'

load_shipping_csv(csv_path)
print('shipping data has been loaded successfully')

shipping data has been loaded successfully


In [78]:
def load_inventory_csv(csv_path):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
            '''INSERT INTO jiggy.inventory (inventory_id, Carrier, EstimatedDelivery, ActualDelivery,
                DeliveryStatus, ProductReturnDate, ReturnReason, RefundAmount,
                CustomerSupportContact, PromotionApplied, CouponCode,
                MembershipStatus, ReviewComment, ReviewDate, FollowUpAction) 
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)''', 
               row 
               )
        
    conn.commit()
    cursor.close()
    conn.close()

In [79]:
csv_path = r'C:\Users\HP\Desktop\New_class\case_study\dataset\inventory.csv'

load_inventory_csv(csv_path)
print('inventory data has been loaded successfully')

inventory data has been loaded successfully


In [83]:
def load_sales_csv(csv_path):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
            '''INSERT INTO jiggy.sales (sales_id, customer_id, shipping_id, inventory_id, product_id,
               Quantity, UnitPrice, TotalPrice, PaymentType, Discount) 
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)''', 
               row 
               )
        
    conn.commit()
    cursor.close()
    conn.close()

In [ ]:
csv_path = r'C:\Users\HP\Desktop\New_class\case_study\dataset\sales.csv'

load_sales_csv(csv_path)
print('sales data has been loaded successfully')